# Notes

## total cars in a county

## total population in a county
## total land area in a county

## vehicles per household

## per cent of commute method per county used

## commute time per means of transportation

## for each graph, have an amalgamated total population and total hours spent on commuting

## the data is processed grouped by population density and then averaged per normalized weighing by total population



# Data Prep

In [1]:

# Init
# choropleth
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# NOTE - THIS TAKES AROUND 13 MINUTES

# OLD # skips the 2nd row that contains indexing names
# Now looking at the row that is consistent between 2009 and 2019 data, using the indexing csv file for guidance.
df_2014 = pd.read_csv('./acs_transportation_data/2014_5yr.csv', na_values=".", skiprows=[0])

In [4]:
df_2014.head()

,Geo_FIPS,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,...,PCT_ACS14_5yr_B08303004,PCT_ACS14_5yr_B08303005,PCT_ACS14_5yr_B08303006,PCT_ACS14_5yr_B08303007,PCT_ACS14_5yr_B08303008,PCT_ACS14_5yr_B08303009,PCT_ACS14_5yr_B08303010,PCT_ACS14_5yr_B08303011,PCT_ACS14_5yr_B08303012,PCT_ACS14_5yr_B08303013
0,1001,"Autauga County, Alabama","Autauga County, Alabama",al,50,0,ACSSF,13,NaN,NaN,...,9.24,13.83,14.38,9.57,21.51,4.34,3.26,6.13,3.29,2.49
1,1003,"Baldwin County, Alabama","Baldwin County, Alabama",al,50,0,ACSSF,14,NaN,NaN,...,15.28,15.09,13.56,6.00,15.06,2.00,3.18,9.63,6.04,2.14
2,1005,"Barbour County, Alabama","Barbour County, Alabama",al,50,0,ACSSF,15,NaN,NaN,...,15.92,13.01,9.57,4.42,14.25,3.38,2.71,8.06,5.13,2.80
3,1007,"Bibb County, Alabama","Bibb County, Alabama",al,50,0,ACSSF,16,NaN,NaN,...,17.27,9.15,9.42,5.19,16.66,5.66,4.70,13.78,5.51,1.33
4,1009,"Blount County, Alabama","Blount County, Alabama",al,50,0,ACSSF,17,NaN,NaN,...,8.53,7.66,8.28,5.22,14.86,4.51,5.66,18.67,12.31,3.17


In [3]:
df_2014['population_density'] = df_2014['

,Geo_FIPS,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,...,PCT_ACS14_5yr_B08303004,PCT_ACS14_5yr_B08303005,PCT_ACS14_5yr_B08303006,PCT_ACS14_5yr_B08303007,PCT_ACS14_5yr_B08303008,PCT_ACS14_5yr_B08303009,PCT_ACS14_5yr_B08303010,PCT_ACS14_5yr_B08303011,PCT_ACS14_5yr_B08303012,PCT_ACS14_5yr_B08303013
0,1001,"Autauga County, Alabama","Autauga County, Alabama",al,50,0,ACSSF,13,NaN,NaN,...,9.24,13.83,14.38,9.57,21.51,4.34,3.26,6.13,3.29,2.49
1,1003,"Baldwin County, Alabama","Baldwin County, Alabama",al,50,0,ACSSF,14,NaN,NaN,...,15.28,15.09,13.56,6.00,15.06,2.00,3.18,9.63,6.04,2.14
2,1005,"Barbour County, Alabama","Barbour County, Alabama",al,50,0,ACSSF,15,NaN,NaN,...,15.92,13.01,9.57,4.42,14.25,3.38,2.71,8.06,5.13,2.80
3,1007,"Bibb County, Alabama","Bibb County, Alabama",al,50,0,ACSSF,16,NaN,NaN,...,17.27,9.15,9.42,5.19,16.66,5.66,4.70,13.78,5.51,1.33
4,1009,"Blount County, Alabama","Blount County, Alabama",al,50,0,ACSSF,17,NaN,NaN,...,8.53,7.66,8.28,5.22,14.86,4.51,5.66,18.67,12.31,3.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72145,"Vega Baja Municipio, Puerto Rico","Vega Baja Municipio, Puerto Rico",pr,50,0,ACSSF,86,NaN,NaN,...,14.46,20.32,9.57,4.22,13.11,1.30,6.28,7.02,14.11,6.22
3216,72147,"Vieques Municipio, Puerto Rico","Vieques Municipio, Puerto Rico",pr,50,0,ACSSF,87,NaN,NaN,...,43.95,32.91,3.88,1.94,6.25,0.00,0.00,0.00,1.39,0.00
3217,72149,"Villalba Municipio, Puerto Rico","Villalba Municipio, Puerto Rico",pr,50,0,ACSSF,88,NaN,NaN,...,11.81,11.86,11.76,3.29,18.19,1.69,6.86,7.56,8.75,1.90
3218,72151,"Yabucoa Municipio, Puerto Rico","Yabucoa Municipio, Puerto Rico",pr,50,0,ACSSF,89,NaN,NaN,...,5.67,14.51,15.17,5.48,18.93,3.13,3.84,7.59,10.16,3.21
